In [1]:
using MacroEnergy
using MacroEnergy: get_edges, can_retrofit, is_retrofit, new_capacity, retired_capacity
using Gurobi
using JuMP

system = MacroEnergy.load_system(@__DIR__)
model = MacroEnergy.generate_model(system)

┌ Info: Loading system from /Users/al3792/Documents/MacroEnergy.jl/ExampleSystems/cement_retrofit_locations_test/system_data.json
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/load_inputs/load_system.jl:18
┌ Info: Loading system data
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/load_inputs/load_system_data.jl:18
┌ Info: Done loading system data. It took 0.42 seconds
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/load_inputs/load_system_data.jl:26
┌ Info: Generating system
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/load_inputs/generate_system.jl:18
┌ Warning: TotalHoursModeled not found in time_data.json - Using 8760 as default value for TotalHoursModeled
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/load_inputs/load_time_data.jl:79
┌ Warning: Using default weights = 1 as no period map provided and each period maps to itself
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/load_inputs/load_time_data.jl:194
┌ W

A JuMP Model
├ solver: none
├ objective_sense: MIN_SENSE
│ └ objective_function_type: AffExpr
├ num_variables: 551927
├ num_constraints: 1401674
│ ├ AffExpr in MOI.EqualTo{Float64}: 271561
│ ├ AffExpr in MOI.GreaterThan{Float64}: 78840
│ ├ AffExpr in MOI.LessThan{Float64}: 516867
│ ├ VariableRef in MOI.EqualTo{Float64}: 13
│ └ VariableRef in MOI.GreaterThan{Float64}: 534393
└ Names registered in the model
  └ :eFixedCost, :eRetrofitCapByRetroId, :eRetrofittedCapByRetroId, :eVariableCost, :vREF

In [2]:
function get_unique_retrofit_tuples(system::MacroEnergy.System)
    # Returns a list of retrofit clusters in the system, described as tuples: (retrofit_id, location)
    can_retrofit_ids = []
    is_retrofit_ids = []
    for e in get_edges(system)
        if can_retrofit(e)
            push!(can_retrofit_ids, (e.retrofit_id, e.location))
        end
        if is_retrofit(e)
            push!(is_retrofit_ids, (e.retrofit_id, e.location))
        end
    end
    @assert unique(can_retrofit_ids) == unique(is_retrofit_ids)
    retrofit_ids = unique(can_retrofit_ids)
    return retrofit_ids
end

function get_can_retrofit_edges(edges, retrofit_id, location)
    # Returns the edges that can be retrofitted with a given retrofit_id and location
    can_retrofit_edges = []
    for edge in edges
        if (can_retrofit(edge) == true) && (edge.retrofit_id == retrofit_id) && (edge.location == location)
            push!(can_retrofit_edges, edge)
        end
    end
    return can_retrofit_edges
end

function get_is_retrofit_edges(edges, retrofit_id, location)
    # Returns the edges that can retrofit other edges with a given retrofit_id and location
    is_retrofit_edges = []
    for edge in edges
        if (is_retrofit(edge) == true) && (edge.retrofit_id == retrofit_id) && (edge.location == location)
            push!(is_retrofit_edges, edge)
        end
    end
    return is_retrofit_edges
end

get_is_retrofit_edges (generic function with 1 method)

In [9]:
edges = get_edges(system)
retrofit_tuples = get_unique_retrofit_tuples(system)

3-element Vector{Any}:
 (1, "MA")
 (1, "CT")
 (1, "ME")

In [8]:
@constraint(model, cRetrofitCapacity[(retrofit_id, retrofit_location) in retrofit_tuples],
    sum(new_capacity(e) for e in get_can_retrofit_edges(edges, retrofit_id, retrofit_location)) <=
    sum(retired_capacity(e) for e in get_is_retrofit_edges(edges, retrofit_id, retrofit_location))
)

1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, Any[(1, "MA"), (1, "CT"), (1, "ME")]
And data, a 3-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape}}:
 cRetrofitCapacity[(1, "MA")] : 114.15 vNEWUNIT_MA_cement_cement_edge - 114.15 vRETUNIT_MA_cement_cement_edge ≤ 0
 cRetrofitCapacity[(1, "CT")] : 114.15 vNEWUNIT_CT_cement_cement_edge - 114.15 vRETUNIT_CT_cement_cement_edge ≤ 0
 cRetrofitCapacity[(1, "ME")] : 114.15 vNEWUNIT_ME_cement_cement_edge - 114.15 vRETUNIT_ME_cement_cement_edge ≤ 0